### Streaming output and events

In [1]:
from helpers import llm
import os, getpass
from llama_index.tools.brave_search import BraveSearchToolSpec

if not os.environ.get("BRAVESEARCH_API_KEY"):
    os.environ["BRAVESEARCH_API_KEY"]=getpass.getpass("BRAVESEARCH_API_KEY: ")

search_tool=BraveSearchToolSpec(api_key=os.environ["BRAVESEARCH_API_KEY"])

In [2]:
from llama_index.core.agent.workflow import AgentWorkflow

from llama_index.core.agent.workflow import (
    AgentInput,
    AgentOutput,
    ToolCallResult,
    AgentStream,
)

workflow = AgentWorkflow.from_tools_or_functions(
    search_tool.to_tool_list(),
    llm=llm,
    system_prompt="You're a helpful assistant that can search the web for information."
)

handler = workflow.run(user_msg="What's the weather like in San Francisco?")

# handle streaming output
async for event in handler.stream_events():
    if isinstance(event, AgentStream):
        print(event.delta, end="", flush=True)
    elif isinstance(event, AgentInput):
        print("Agent input: ", event.input)  # the current input messages
        print("Agent name:", event.current_agent_name)  # the current agent name
    elif isinstance(event, AgentOutput):
        print("Agent output: ", event.response)  # the current full response
        print("Tool calls made: ", event.tool_calls)  # the selected tool calls, if any
        print("Raw LLM response: ", event.raw)  # the raw llm api response
    elif isinstance(event, ToolCallResult):
        print("Tool called: ", event.tool_name)  # the tool name
        print("Arguments to the tool: ", event.tool_kwargs)  # the tool kwargs
        print("Tool output: ", event.tool_output)  # the tool output            

# print final output
print(str(await handler))

Agent input:  [ChatMessage(role=<MessageRole.SYSTEM: 'system'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text="You're a helpful assistant that can search the web for information.")]), ChatMessage(role=<MessageRole.USER: 'user'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text="What's the weather like in San Francisco?")])]
Agent name: Agent
Let me search for current weather information in San Francisco.Agent output:  assistant: Let me search for current weather information in San Francisco.
Tool calls made:  [ToolSelection(tool_id='toolu_vrtx_01NUfxfvQ5ievV5mrBaEaaU1', tool_name='brave_search', tool_kwargs={'query': 'current weather San Francisco'})]
Raw LLM response:  {'id': 'gen-1753735935-YEpuEphMmrzQnzX7cKC5', 'choices': [{'delta': {'content': '', 'function_call': None, 'refusal': None, 'role': 'assistant', 'tool_calls': None}, 'finish_reason': None, 'index': 0, 'logprobs': None, 'native_finish_reason': None}], 'created': 1753735935, 'model': 'anth